## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
import numpy as np
import random
import geopandas as gpd
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

df_posicoes = load_csv('df_posicoes.csv')

In [3]:
gdf = gpd.GeoDataFrame(df_posicoes, geometry=gpd.points_from_xy(df_posicoes.lon, df_posicoes.lat), crs="EPSG:4326")

gdf = gdf.to_crs("EPSG:31983")

gdf

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo,geometry
0,917H-10,11897,2025-09-23 10:19:04-03:00,-23.486367,-46.726082,False,Padron,POINT (323730.274 7401579.573)
1,917H-10,11886,2025-09-23 10:18:57-03:00,-23.589215,-46.634663,False,Articulado (18m),POINT (333197.69 7390299.543)
2,917H-10,11882,2025-09-23 10:18:59-03:00,-23.589249,-46.633758,True,Básico,POINT (333290.098 7390296.777)
3,917H-10,11718,2025-09-23 10:19:04-03:00,-23.535546,-46.726671,True,Articulado (23m),POINT (323735.607 7396132.588)
4,917H-10,11903,2025-09-23 10:19:36-03:00,-23.486014,-46.725750,False,Articulado (18m),POINT (323763.665 7401619.016)
...,...,...,...,...,...,...,...,...
74789,6813-10,72019,2025-09-23 10:25:40-03:00,-23.662937,-46.762801,True,Básico,POINT (320220.391 7381980.034)
74790,4008-21,48243,2025-09-23 10:26:04-03:00,-23.579354,-46.432372,True,Articulado (18m),POINT (353831.801 7391612.589)
74791,4086-10,47390,2025-09-23 10:25:43-03:00,-23.583050,-46.407673,True,Articulado (18m),POINT (356356.665 7391228.325)
74792,3010-10,36189,2025-09-23 10:25:47-03:00,-23.513862,-46.447780,False,Midiônibus,POINT (352186.022 7398848.798)


In [4]:
gdf["x"] = gdf.geometry.x
gdf["y"] = gdf.geometry.y

gdf.head()

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo,geometry,x,y
0,917H-10,11897,2025-09-23 10:19:04-03:00,-23.486367,-46.726082,False,Padron,POINT (323730.274 7401579.573),323730.273665,7.401580e+06
1,917H-10,11886,2025-09-23 10:18:57-03:00,-23.589215,-46.634663,False,Articulado (18m),POINT (333197.69 7390299.543),333197.689891,7.390300e+06
2,917H-10,11882,2025-09-23 10:18:59-03:00,-23.589249,-46.633758,True,Básico,POINT (333290.098 7390296.777),333290.097553,7.390297e+06
3,917H-10,11718,2025-09-23 10:19:04-03:00,-23.535546,-46.726671,True,Articulado (23m),POINT (323735.607 7396132.588),323735.606933,7.396133e+06
4,917H-10,11903,2025-09-23 10:19:36-03:00,-23.486014,-46.725750,False,Articulado (18m),POINT (323763.665 7401619.016),323763.665187,7.401619e+06


In [5]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]

consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [6]:
def get_tipo_onibus(id_onibus):

    onibus = df_posicoes[df_posicoes['id_onibus']==id_onibus]

    return onibus['modelo'].unique()[0]

In [7]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [8]:
#df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

In [9]:
gdf['id_onibus'].nunique()

10760

In [10]:
data_frame = []

#for id_onibus in gdf['id_onibus'].unique():
    #df_onibus = gdf[gdf['id_onibus'] == id_onibus].reset_index(drop=True)
    #df_onibus.sort_values(by='timestamp', inplace=True)

for id_onibus in gdf['id_onibus'].unique():
    df_onibus = (gdf[gdf['id_onibus'] == id_onibus].sort_values(by='timestamp').reset_index(drop=True))

    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['timestamp']
            momento_final = df_onibus.loc[i+1, 'timestamp']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            dist = np.sqrt((x_depois - x_atual)**2 + (y_depois - y_atual)**2)
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'linha' : row['linha'],
                'is_eletrico' : row['is_eletrico'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

gdf_final = pd.DataFrame(data_frame)
gdf_final.head()

,id_onibus,linha,is_eletrico,distancia_percorrida,ponto_inicial,ponto_final,momento_inicial,momento_final,modelo,litro_diesel_km,emissao_no2,emissao_co2
0,11897,917H-10,False,10.601070,"[323730.2736653163, 7401579.57348449]","[323723.320928209, 7401571.570852276]",2025-09-23 10:19:04-03:00,2025-09-23 10:19:48-03:00,Padron,0.63,0.228093,0.540298
1,11897,917H-10,False,62.279468,"[323723.320928209, 7401571.570852276]","[323763.6651873907, 7401619.016325403]",2025-09-23 10:19:48-03:00,2025-09-23 10:22:45-03:00,Padron,0.63,1.340005,3.174161
2,11897,917H-10,False,0.000000,"[323763.6651873907, 7401619.016325403]","[323763.6651873907, 7401619.016325403]",2025-09-23 10:22:45-03:00,2025-09-23 10:23:30-03:00,Padron,0.63,0.000000,0.000000
3,11897,917H-10,False,0.000000,"[323763.6651873907, 7401619.016325403]","[323763.6651873907, 7401619.016325403]",2025-09-23 10:23:30-03:00,2025-09-23 10:24:14-03:00,Padron,0.63,0.000000,0.000000
4,11897,917H-10,False,51.679120,"[323763.6651873907, 7401619.016325403]","[323730.2736653163, 7401579.57348449]",2025-09-23 10:24:14-03:00,2025-09-23 10:24:58-03:00,Padron,0.63,1.111928,2.633899


In [14]:
gdf_final['momento_inicial'] = pd.to_datetime(gdf_final['momento_inicial'])

In [15]:
dados_trips_layer = []

for onibus in gdf_final['id_onibus'].unique():

    df_onibus = gdf_final[gdf_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    for i, row in df_onibus.iterrows():
        coordinates.append(row['ponto_inicial'])
        stamp = datetime.fromisoformat(row['momento_inicial'].isoformat())
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']

    dados_trips_layer.append({
        'id_onibus': onibus,
        'linha': row['linha'],
        'emissao_co2': emissao_total,
        'is_eletrico': row['is_eletrico'],
        'coordinates' : coordinates,
        'timestamps' : timestamps
    })


df_trips = pd.DataFrame(dados_trips_layer)

In [16]:
def init_on_zero(timestamp_list)->list[float]:
    
    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts.total_seconds()) for ts in new_timestamps]

df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [17]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [18]:
eval(coord_t)

[[323730.2736653163, 7401579.57348449],
 [323723.320928209, 7401571.570852276],
 [323763.6651873907, 7401619.016325403],
 [323763.6651873907, 7401619.016325403],
 [323763.6651873907, 7401619.016325403],
 [323730.2736653163, 7401579.57348449]]

In [19]:
df_trips.loc[0, 'timestamps']

[0, 44, 221, 266, 310, 354]

In [20]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [21]:
save_csv(gdf_final, 'gdf_final.csv')

save_csv(df_trips, 'df_trips.csv')

Base salva em data\gdf_final.csv
Base salva em data\df_trips.csv
